In [6]:
import cadCAD
import numpy as np


max_players = 50
max_users_per_player = 50
max_stamps_per_user = 80
stamp_cost_range = (0, 1000)
player_funds = 10000
grant_pool = 100000
max_grants = 20

# Initialize the system
initial_conditions = {'players': [], 'users': [], 'identities': [], 'grants': [], 'player_funds': player_funds, 'grant_pool': grant_pool, 'allocated_funds': 0, 'matching_funds': 0}

import random

# Set the number of players and users
num_players = 50
num_users = 50

# Initialize the lists of players, users, and identities
players = []
users = []
identities = []

# Create the players and add them to the list
for i in range(num_players):
    player = {'id': i, 'users': []}
    players.append(player)

# Create the users and identities and add them to the appropriate lists
for i in range(num_users):
    user = {'id': i, 'player_id': random.randint(0, num_players-1),'grant': None, 'sybil_coefficient':0}
    users.append(user)
    player = [p for p in players if p['id'] == user['player_id']][0]
    player['users'].append(user)
    
    num_identities = random.randint(0, 50)
    for j in range(num_identities):
        identity = {'user_id': user['id'], 'trust_score': random.randint(0, 1000)/100}
        identities.append(identity)

# Generate a list of 100 stamps with random costs and trust scores
identities = [{'cost': random.randint(0, 1000), 'trust_score': random.randint(0, 100)} for _ in range(100)]

# Generate a list of 20 grants with random donations from random users
grants = [{'donations': [{'user_id': random.randint(1, 50), 'amount': random.uniform(0, 1000)} for _ in range(random.randint(1, 50))], 'allocated_funds': 0, 'matching_funds': 0} for _ in range(20)]


# Actions
actions = []
#user['grant'] = None
#user['sybil_coefficient'] = None

# Create players
for i in range(max_players):
    player = {'id': i, 'users': [], 'player_funds': player_funds, 'sybil_coefficient': 0}
    # Create users for each player
    for j in range(max_users_per_player):
        user = {'id': j, 'player_id': i, 'identities': [], 'grant': None, 'sybil_coefficient': 0}
        player['users'].append(user)
        # Create identities for each user
        for k in range(max_stamps_per_user):
            stamp_cost = np.random.randint(*stamp_cost_range)
            trust_score = stamp_cost / 100
            identity= {'id': k, 'user_id': j, 'player_id': i, 'stamp_cost': stamp_cost, 'trust_score': trust_score}
user['identities'].append(identity)
initial_conditions['players'].append(player)



def update(params, step, sL, s, users, grant):
    # Unpack system variables
    players = s['players']
    identities = s['identities']
    grants = s['grants']
    player_funds = s['player_funds']
    grant_pool = s['grant_pool']
    allocated_funds = s['allocated_funds']
    matching_funds = s['matching_funds']
    
    # Calculate sybil coefficient for each user
    for user in users:
        user_identities = [i for i in identities if i['user_id'] == user['id']]
        user['sybil_coefficient'] = 0
        user['sybil_coefficient'] = sum(i['trust_score'] for i in user_identities)
        
        # Check if user should be squelched
        player = [p for p in players if p['id'] == user['player_id']][0]
        if len(user_identities) < 10 or user['sybil_coefficient'] < 20:
            user['sybil_coefficient'] = 0
            if len(player['users']) > 1:
                probability_of_squelch = 0.8
            else:
                probability_of_squelch = 0.2
            if np.random.rand() < probability_of_squelch:
                user['sybil_coefficient'] = 0




# Check if any new grants should be created

for player in players:
    for user in player['users']:
        if user['grant'] is None and user['sybil_coefficient'] is not None and user['sybil_coefficient'] > 0 and len(grants) < max_grants:
        #if user['grant'] is None and user['sybil_coefficient'] > 0 and len(grants) < max_grants:
            grant = {'id': len(grants), 'user_id': user['id'], 'player_id': player['id'], 'donors': [], 'allocated_funds': 0, 'matching_funds': 0,
'total_unique_donors': 0
}
grants.append(grant)
#user['grant'] = grant


#Accept donations to grants
for player in players:
    for user in player['users']:
        if user['sybil_coefficient'] > 0 and user['grant'] is not None:
            grant = user['grant']
            if grant is not None:
                donation_amount = np.random.randint(0, player_funds[player['id']]+1)
                player_funds[player['id']] -= donation_amount
                if 'allocated_funds' not in grant:
                    grant['allocated_funds'] = 0
                    grant['allocated_funds'] += donation_amount * np.sqrt(user['sybil_coefficient'])
                    grant['total_unique_donors'] += 1
                    

# Calculate matching funds for each grant
for grant in grants:
    if 'allocated_funds' in grant:
        grant['matching_funds'] = grant_pool * (grant['allocated_funds'] / sum(g['allocated_funds'] for g in grants))


#Distribute rewards to players
for player in players:
    for user in player['users']:
        if user['grant'] is not None:
            grant = user['grant']
player_funds[player['id']] += grant['matching_funds'] * 0.5
user['grant'] = None
# Distribute remaining matching funds to donors
for donor_player in players:
    for donor_user in donor_player['users']:
        if donor_user['grant'] == grant['id']:
            donor_player['funds'] += grant['matching_funds'] * 0.5 * donor_user['sybil_coefficient'] / grant['allocated_funds']
grant_pool -= grant['matching_funds']

#Print final results
print("Grants:")
for grant in grants:
    print(f"Grant {grant['id']}: Matching funds = {grant['matching_funds']}, Total unique donors = {grant['total_unique_donors']}")
print("\nPlayers:")
for player in players:
    print(f"Player {player['id']}: Funds = {player_funds[player['id']]}, Sybil coefficient = {player['sybil_coefficient']}")
    


NameError: name 'grant' is not defined